In [1]:
import keras
from keras import models
from keras import layers
from keras.datasets import reuters
from keras.utils.np_utils import to_categorical
import numpy as np
from sklearn.model_selection import KFold

np.set_printoptions(precision=4)

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# import the reuters data
(input_train_data, input_train_labels), (input_test_data, input_test_labels) = reuters.load_data(num_words=10000)

In [3]:
# define a function to one hot encode training data
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [4]:
# Our vectorized training data
x_train = vectorize_sequences(input_train_data)

In [5]:
# create the one hot train labels
one_hot_train_labels = to_categorical(input_train_labels)

In [6]:
# define a function to build a compiled model
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(46, activation='softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [7]:
# get a compiled model
model = build_model()

In [8]:
# create a cross validation object from sci-kit learn
kf = KFold(n_splits = 4)

In [9]:
# create an empty list to store the accuracy data
accuracy_list = []

In [10]:
# for each train / test fold in the cross validation object
for train_indices, test_indices in kf.split(x_train):
    
    # use the train / test indices in the fold to create the train and test sets
    x_train_fold = x_train[train_indices]
    x_test_fold = x_train[test_indices]
    y_train_fold = one_hot_train_labels[train_indices]
    y_test_fold = one_hot_train_labels[test_indices]

    # train the model using the training data fold
    history = model.fit(x_train_fold, y_train_fold,epochs=7, batch_size=512, verbose = 0)

    # test the model using the test data fold
    loss, accuracy = model.evaluate(x_test_fold, y_test_fold, verbose = 0)
    print("accuracy = ", accuracy)
    
    accuracy_list.append(accuracy)

accuracy =  0.7422083704363313
accuracy =  0.8860195903298267
accuracy =  0.9389755011135857
accuracy =  0.9363028953494898


In [11]:
# print the final average accuracy
print("average accuracy = ", np.mean(np.array(accuracy_list)))

average accuracy =  0.8758765893073084
